## Accessing AIStore objects with the SDK

In [ ]:
pip install aistore

### Use the client class to get a reference to a named bucket:

In [ ]:
from aistore import Client
from pathlib import Path

ais_url = "http://localhost:8080"
client = Client(ais_url)
bucket = client.bucket("my-bck")

### First, create a few objects

In [ ]:
object_names = [f"example_obj_{i}" for i in range(10)]
for name in object_names:
    bucket.object(name).put_content(b"object content")
# Create one with a different prefix
bucket.object("prefix-example").put_content(b"prefixed object content")

## There are 3 ways to list the objects inside a bucket
1. list_objects provides a page of objects and the data to get the next page
2. list_objects_iter returns an iterator over all objects
3. list_all_objects returns a single list of all objects (calling list_objects until exhausted)

### list_objects

In [ ]:
objects = []
response = bucket.list_objects(page_size=3)
objects.extend(response.entries)
# Use the info from the previous response to get the next page of objects
bucket.list_objects(
    uuid=response.uuid, continuation_token=response.continuation_token, page_size=3
)
objects.extend(response.entries)
# Now we should have the first 2 pages of size 3 each
print(objects)
print("Total number of objects:", len(objects))

### list_objects_iter

In [ ]:
iterator = bucket.list_objects_iter(prefix="example_", props="name,size,copies")
for bucket_entry in iterator:
    print(bucket_entry)

### list_all_objects

In [ ]:
bucket.list_all_objects(prefix="example_")

## Call head() on an object to view properties

In [ ]:
bucket.object(object_names[0]).head()

## Object.get returns an ObjStream reader with a few ways to access the data

In [ ]:
stream = bucket.object(object_names[0]).get()
print("Read all from the stream:", stream.read_all())
print("Raw response:", stream.raw())

### For larger objects, you can read in chunks

In [ ]:
print("Chunks:")
for chunk in bucket.object(object_names[0]).get(chunk_size=3):
    print(chunk)

### Or you can provide your own writer

In [ ]:
# Pass a writer that appends to a file
filename = Path().absolute().joinpath("example-obj-writer")
with open(filename, "ab") as writer:
    bucket.object(object_names[0]).get(writer=writer)

# Read from the file to see the output
with open(filename, "rb") as reader:
    print(reader.read())

filename.unlink()

## Working with multiple objects
AIS supports multi-object operations on groups of objects. An `ObjectGroup` can be created with one of:
- a list of object names
- an [ObjectRange](https://github.com/NVIDIA/aistore/blob/master/python/aistore/sdk/object_range.py)
- a string template.

### Create Object Group by list of names

In [ ]:
my_objects = client.bucket("my-ais-bucket").objects(obj_names=object_names)

### Create Object Group by ObjectRange

In [ ]:
from aistore.sdk.object_range import ObjectRange

my_object_range = ObjectRange(prefix="example_obj_", min_index=1, max_index=3)
my_objects = client.bucket("my-ais-bucket").objects(obj_range=my_object_range)

### Create Object Group by Template String
String templates can be passed directly to AIS following the [syntax described here](https://github.com/NVIDIA/aistore/blob/master/docs/batch.md#operations-on-multiple-selected-objects)

In [ ]:
# Equivalent to the range above
my_object_template = "example_obj_{1..3}"
my_objects = bucket.objects(obj_template=my_object_template)
# More advanced template example with multiple ranges and defined steps
complex_range = "example_obj_{0..10..2}_details_{1..9..2}.file-extension"

### Prefetch, Evict, and Delete with multiple Objects

In [ ]:
# Prefetch and Evict only work for cloud backends
my_objects.prefetch()
my_objects.evict()

In [ ]:
delete_job_id = my_objects.delete()
client.job(delete_job_id).wait()